In [1]:
from datetime import datetime, timedelta
from typing import List, Dict, Optional

class Book:
    def __init__(self, title: str, author: str, description: str, pub_date: datetime, isbn: str, copies_available: int, checked_out_by: Optional[List[str]] = None):
        if checked_out_by is None:
            checked_out_by = []
        self.title = title
        self.author = author
        self.description = description
        self.pub_date = pub_date
        self.isbn = isbn
        self.copies_available = copies_available
        self.checked_out_by = checked_out_by
        self.available = self.copies_available > 0
        self.waiting_list = []
        
    def __str__(self):
        return f"'{self.title}' by {self.author} - ISBN: {self.isbn} - Copies Available: {self.copies_available}"

    def checkout(self, name: str):
        if self.copies_available > 0:
            self.checked_out_by.append(name)
            self.copies_available -= 1
            self.available = self.copies_available > 0
        else:
            raise ValueError(f"No copies of '{self.title}' are available to checkout.")

    def checkin(self, name: str):
        if name in self.checked_out_by:
            self.checked_out_by.remove(name)
            self.copies_available += 1
            self.available = True
        else:
            raise ValueError(f"{name} did not check out '{self.title}'.")

    def add_copies(self, number: int):
        self.copies_available += number

    def remove_copies(self, number: int):
        if number <= self.copies_available:
            self.copies_available -= number
        else:
            raise ValueError(f"Cannot remove {number} copies. Only {self.copies_available} copies are available.")

    def add_waitinglist(self, member):
        self.waiting_list.append(member)

    def notify_nextmember(self):
        if self.waiting_list:
            bordering_member = self.waiting_list.pop(0)
            bordering_member.recive_notofication(f" '{self.title}' is now available for brroeing.")
            return bordering_member                                     

class Member:
    def __init__(self, name: str, member_id: str):
        self.name = name
        self.member_id = member_id
        self.library_card = LibraryCard(self) 
        self.damages = 0

    def borrow_book(self, book: Book):
        if book.available and book.copies_available > 0:
            self.library_card.borrow_book(book)
            book.checkout(self.name)
            print(f"{self.name} has borrowed '{book.title}'.")
        else:
            raise ValueError(f"'{book.title}' is not available for borrowing.")

    def return_book(self, book: Book):
        if book in self.library_card.borrowing_history:
            self.library_card.return_book(book)
            book.checkin(self.name)
            print(f"{self.name} has returned '{book.title}'.")
        else:
            raise ValueError(f"{self.name} has not borrowed '{book.title}'.")

    def display_borrowing_history(self):
        self.library_card.display_borrowing_history()


class Librarian:
    def __init__(self, name: str):
        self.name = name

    def add_book(self, library: 'Library', book: Book):
        library.add_book(book)
        print(f"{self.name} added '{book.title}'.\n")

    def remove_book(self, library: 'Library', book: Book):
        library.remove_book(book)
        print(f"{self.name} removed '{book.title}'.\n")

    def search_book(self, library: 'Library', title: str) -> Optional[Book]:
        return library.search_book(title)


class Library:
    def __init__(self, name: str):
        self.name = name
        self.booklist: List[Book] = []
        self.lendDict: Dict[Book, str] = {}

    def display_books(self):
        print(f"\n We have the following books in our library '{self.name}':")
        for book in self.booklist:
            print(f"- {book}")

    def lend_book(self, member: Member, book: Book):
        if book in self.lendDict:
            print(f"\n Book '{book.title}' is already being used by {self.lendDict[book]}.")
        else:
            if book in self.booklist:
                self.lendDict[book] = member.name
                self.booklist.remove(book)
                print(f"\n Lender-book database has been updated. '{book.title}' has been lent to {member.name}.")
            else:
                print(f"\n Sorry, the book '{book.title}' is not available in the library.")

    def add_book(self, book: Book):
        self.booklist.append(book)
        print(f"\n Book '{book.title}' has been added to the book list.")

    def remove_book(self, book: Book):
        if book in self.booklist:
            self.booklist.remove(book)
            print(f"\n Book '{book.title}' has been removed from the book list.")
        else:
            print(f"\n Book '{book.title}' is not in the library's book list.")

    def return_book(self, book: Book):
        if book in self.lendDict:
            self.booklist.append(book)
            self.lendDict.pop(book)
            print(f"\n Book '{book.title}' has been returned and added back to the book list.")
        else:
            print(f"\n Book '{book.title}' was not lent out.")

    def search_book(self, title: str) -> Optional[Book]:
        for book in self.booklist:
            if book.title.lower() == title.lower():
                return book
        print(f"\n '{title}' not found in the library.")
        return None


class LibraryCard:
    def __init__(self, member: Member):
        self.member = member
        self.borrowing_history: List[Dict[str, Optional[datetime]]] = []

    def borrow_book(self, book: Book):
        if any(entry['book'] == book and entry['return_date'] is None for entry in self.borrowing_history):
            raise ValueError(f"\n {self.member.name} has already borrowed '{book.title}'.")
        borrow_entry = {
            'book': book,
            'borrow_date': datetime.now(),
            'return_date': None
        }
        self.borrowing_history.append(borrow_entry)
        print(f"\n {self.member.name} borrowed '{book.title}' on {borrow_entry['borrow_date'].strftime('%Y-%m-%d')}.")

    def return_book(self, book: Book):
        for entry in self.borrowing_history:
            if entry['book'] == book and entry['return_date'] is None:
                entry['return_date'] = datetime.now()
                print(f"\n {self.member.name} returned '{book.title}' on {entry['return_date'].strftime('%Y-%m-%d')}.")
                return
        raise ValueError(f"\n {self.member.name} did not borrow '{book.title}' or it was already returned.")

    def display_borrowing_history(self):
        print(f"\n Borrowing history for {self.member.name}:")
        for entry in self.borrowing_history:
            book = entry['book']
            borrow_date = entry['borrow_date'].strftime('%Y-%m-%d')
            return_date = entry['return_date'].strftime('%Y-%m-%d') if entry['return_date'] else "Not returned yet"
            print(f"- '{book.title}' (Borrowed on: {borrow_date}, Returned on: {return_date})\n")


# Create a library
library = Library(name="Central Library")

# Initialize books
book1 = Book(
    title="1984",
    author="George Orwell",
    description="A dystopian novel set in a totalitarian regime, exploring themes of surveillance, government control, and individual freedom.",
    pub_date=datetime(1949, 6, 8),
    isbn="9780451524935",
    copies_available=4
)

book2 = Book(
    title="Pride and Prejudice",
    author="Jane Austen",
    description="A classic novel that explores the themes of love, reputation, and class in early 19th century England, centered around Elizabeth Bennet and Mr. Darcy.",
    pub_date=datetime(1813, 1, 28),
    isbn="9780141040349",
    copies_available=6
)

# Create a librarian
librarian = Librarian(name="John")

# Librarian adds books to the library
librarian.add_book(library, book1)
librarian.add_book(library, book2)

# Display books in the library
library.display_books()

# Initialize two members
member1 = Member(name="Malathi", member_id="MP004")
member2 = Member(name="Raj", member_id="MP002")

# Member1 (Malathi) borrows a book
try:
    member1.borrow_book(book1)
except ValueError as e:
    print(e)

# Display member1's borrowing history
member1.display_borrowing_history()

# Member1 tries to borrow the same book again (should raise an error)
try:
    member1.borrow_book(book1)
except ValueError as e:
    print(e)

# Member1 returns a book
try:
    member1.return_book(book1)
except ValueError as e:
    print(e)

# Display member1's borrowing history after returning the book
member1.display_borrowing_history()

# Member2 (Raj) borrows a different book
try:
    member2.borrow_book(book2)
except ValueError as e:
    print(e)

# Display member2's borrowing history
member2.display_borrowing_history()

# Member2 tries to borrow a different book
try:
    member2.borrow_book(book2)
except ValueError as e:
    print(e)

# Member2 returns a book
try:
    member2.return_book(book2)
except ValueError as e:
    print(e)

# Display member2's borrowing history after returning the book
member2.display_borrowing_history()

# Librarian removes a book from the library (using member2)
try:
    librarian.remove_book(library, book2)
except ValueError as e:
    print(e)

# Display available books after removing a book
library.display_books()

# Search for a book in the library
searched_book = library.search_book("The Great Gatsby")
if searched_book:
    print(f"Found: {searched_book}")


 Book '1984' has been added to the book list.
John added '1984'.


 Book 'Pride and Prejudice' has been added to the book list.
John added 'Pride and Prejudice'.


 We have the following books in our library 'Central Library':
- '1984' by George Orwell - ISBN: 9780451524935 - Copies Available: 4
- 'Pride and Prejudice' by Jane Austen - ISBN: 9780141040349 - Copies Available: 6

 Malathi borrowed '1984' on 2024-10-02.
Malathi has borrowed '1984'.

 Borrowing history for Malathi:
- '1984' (Borrowed on: 2024-10-02, Returned on: Not returned yet)


 Malathi has already borrowed '1984'.
Malathi has not borrowed '1984'.

 Borrowing history for Malathi:
- '1984' (Borrowed on: 2024-10-02, Returned on: Not returned yet)


 Raj borrowed 'Pride and Prejudice' on 2024-10-02.
Raj has borrowed 'Pride and Prejudice'.

 Borrowing history for Raj:
- 'Pride and Prejudice' (Borrowed on: 2024-10-02, Returned on: Not returned yet)


 Raj has already borrowed 'Pride and Prejudice'.
Raj has not borrowed 'Pr